# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import re

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [8]:
orders = pd.read_csv('./Orders.csv', sep=",")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [49]:
#Sub Problem 1: we create a new data frame based in "orders" but grouped by "CustomerID".
# Then we aggregate by the sum of the "amount_spent":

customers = pd.DataFrame()

customers["Amount spent"] = orders.groupby("CustomerID").agg('sum')["amount_spent"]
customers= customers.reset_index()
customers.head()

,CustomerID,Amount spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [50]:
# Sub Problem 2: we need to find the 75th - 95th percentile range. We will see where each customer fits depending on
# his/her spend, based on these percentile values:

q75 = customers["Amount spent"].quantile(q=0.75)
q95 = customers["Amount spent"].quantile(q=0.95)

print(f"75th percentile is {q75} and 95th percentile is {q95}")

75th percentile is 1661.6400000000003 and 95th percentile is 5840.181999999983


In [54]:
#Sub Problem 3: now we label each customer by applying a function to the value on the "Amount Spent" column and saving
#the result in an new colum:

def cust_cat(amount):
    if q75 < amount <= q95: return "Preferred"
    if q95 < amount: return "VIP"
    return "Regular"

customers ["Category"] = customers ["Amount spent"].apply(cust_cat)


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [58]:
# First I will create a new dataframe containing unique customer_id-country pairs:
countries = orders.copy()[["CustomerID","Country"]].drop_duplicates()


In [60]:
# Notice that this new df contains duplicated customer_ids, that's propably because these customers (only 8 of them)
# have bouht items in different countries:
countries["CustomerID"].value_counts().head(9)

12455    2
12429    2
12370    2
12457    2
12394    2
12422    2
12431    2
12417    2
14333    1
Name: CustomerID, dtype: int64

In [62]:
# We'll keep these customers as they're not that many. Now we merge the countries df with our customers df to obtain
# the country for each customer_id. Notice this will duplicate those 8 customers in our merged table as well.

merged = customers.merge(countries,how="left",on="CustomerID")
merged.head()

,CustomerID,Amount spent,Category,Country
0,12346,77183.60,VIP,United Kingdom
1,12347,4310.00,Preferred,Iceland
2,12348,1797.24,Preferred,Finland
3,12349,1757.55,Preferred,Italy
4,12350,334.40,Regular,Norway


Now let's see which country has the most VIPs:

In [70]:
top_count = pd.DataFrame()

top_count["VIP"] = merged[merged["Category"]=="VIP"].groupby("Country").agg('count')["CustomerID"]
top_count["Preferred"] = merged[merged["Category"]=="Preferred"].groupby("Country").agg('count')["CustomerID"]
top_count["Sum"] = top_count["VIP"] + top_count["Preferred"]
top_count= top_count.reset_index()
display(top_count.sort_values("VIP",ascending=False).head(1))


,Country,VIP,Preferred,Sum
17,United Kingdom,177,755.0,932.0


UK wins with 177 VIP customers!

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [71]:
display(top_count.sort_values("Sum",ascending=False).head(1))


,Country,VIP,Preferred,Sum
17,United Kingdom,177,755.0,932.0


UK wins again!